In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from pprint import pprint

In [2]:
# 초단기예보 : /getUltraSrtFcst , 단기예보 : /getVilageFcst , 초단기실황 : getUltraSrtNcst
base_url = "http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst"
with open('keys/기상청api.txt') as file:
    service_key = file.read()       # 발급받은 기상청 API 키 입력

In [3]:
def get_weather(service_key, base_date, base_time, nx, ny):
    
    payload = {
        'serviceKey': service_key,
        'dataType': 'JSON',
        'base_date': base_date,  # 예보 일자 (YYYYMMDD)
        'base_time': base_time,  # 예보 시간 (HHMM)
        'nx': nx,  # 예보 지점 x 좌표
        'ny': ny   # 예보 지점 y 좌표
    }

    response = requests.get(base_url, params=payload)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None

In [4]:
# 예제 사용
api_key = service_key
base_date = "20231121"  # 예보 일자 (YYYYMMDD)
base_time = "0800"      # 예보 시간 (HHMM)
nx = "60"               # 예보 지점 x 좌표
ny = "127"              # 예보 지점 y 좌표

weather_data = get_weather(api_key, base_date, base_time, nx, ny)
print(weather_data)

{'response': {'header': {'resultCode': '10', 'resultMsg': '최근 1일 간의 자료만 제공합니다.'}}}


In [5]:
# 웹 요청할 base_date, base_time 계산
now = datetime.now() # 현재 시각 데이터 생성

# 40분 이전이면 현재 시보다 1시간 전 `base_time`을 요청한다.
if now.minute <= 40:
    # 단. 00:40분 이전이라면 `base_date`는 전날이고 `base_time`은 2300이다.
    if now.hour == 0 :
        base_date = (now-timedelta(days=1)).strftime('%Y%m%d')
        base_time = '2300'
    else:
        base_date = now.strftime('%Y%m%d')
        base_time = (now-timedelta(hours=1)).strftime('%H00')
# 40분 이후면 현재 시와 같은 `base_time`을 요청한다.
else:
    base_date = base_date = now.strftime('%Y%m%d') 
    base_time = now.strftime('%H00')
# print(base_date, base_time# 웹 요청시 같이 전달될 데이터 = 요청 메시지
params = {
    'serviceKey' : service_key,
    'numOfRows' : 30,
    'pageNo' : 1,
    'dataType' : 'JSON',
    'base_date' : base_date, # 오늘 날짜
    'base_time' : base_time, # 요청 가능 발표 시간
    'nx' : 62, # 성남시 운중동
    'ny' : 123 
}

In [6]:
res = requests.get(url=base_url , params=params)
print(res.status_code, type(res.text), res.url)
print()
print(res.text)

200 <class 'str'> http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst?serviceKey=X%2B4lIUWK7HKc4dsO1rh5%2BxZfzAg37vdzPTYUT8UmSL0IySMIZxPtcVyyRpaYeNVkZexlJwcxTRwJ%2FrdSUvAZHg%3D%3D&numOfRows=30&pageNo=1&dataType=JSON&base_date=20231204&base_time=1100&nx=62&ny=123

{"response":{"header":{"resultCode":"00","resultMsg":"NORMAL_SERVICE"},"body":{"dataType":"JSON","items":{"item":[{"baseDate":"20231204","baseTime":"1100","category":"PTY","nx":62,"ny":123,"obsrValue":"0"},{"baseDate":"20231204","baseTime":"1100","category":"REH","nx":62,"ny":123,"obsrValue":"35"},{"baseDate":"20231204","baseTime":"1100","category":"RN1","nx":62,"ny":123,"obsrValue":"0"},{"baseDate":"20231204","baseTime":"1100","category":"T1H","nx":62,"ny":123,"obsrValue":"4.3"},{"baseDate":"20231204","baseTime":"1100","category":"UUU","nx":62,"ny":123,"obsrValue":"-0.6"},{"baseDate":"20231204","baseTime":"1100","category":"VEC","nx":62,"ny":123,"obsrValue":"120"},{"baseDate":"20231204","baseTime":"1100",

In [7]:
# 웹 요청할 base_date, base_time 계산
now = datetime.now() # 현재 시각 데이터 생성

# 40분 이전이면 현재 시보다 1시간 전 `base_time`을 요청한다.
if now.minute <= 40:
    # 단. 00:40분 이전이라면 `base_date`는 전날이고 `base_time`은 2300이다.
    if now.hour == 0 :
        base_date = (now-timedelta(days=1)).strftime('%Y%m%d')
        base_time = '2300'
    else:
        base_date = now.strftime('%Y%m%d')
        base_time = (now-timedelta(hours=1)).strftime('%H00')
        
# 40분 이후면 현재 시와 같은 `base_time`을 요청한다.
else:
    base_date = base_date = now.strftime('%Y%m%d') 
    base_time = now.strftime('%H00')
# print(base_date, base_time)


# 웹 요청시 같이 전달될 데이터 = 요청 메시지
params = {
    'serviceKey' : service_key,
    'numOfRows' : 30,
    'pageNo' : 1,
    'dataType' : 'JSON',
    'base_date' : base_date, # '20200611', # 오늘 날짜
    'base_time' : base_time, # '1400', # 요청 가능 발표 시간
    'nx' : 62, # 성남시 운중동
    'ny' : 123 
}

res = requests.get(url=base_url , params=params)
# print(res.status_code, type(res.text), res.url)
# print()
# print(res.text)

# 응답 데이터 정리
from pprint import pprint  # 구조있는 데이터를 더 편하게 보여줌
data = res.json() # json.loads(res.text)와 같은 기능
pprint(data)

{'response': {'body': {'dataType': 'JSON',
                       'items': {'item': [{'baseDate': '20231204',
                                           'baseTime': '1100',
                                           'category': 'PTY',
                                           'nx': 62,
                                           'ny': 123,
                                           'obsrValue': '0'},
                                          {'baseDate': '20231204',
                                           'baseTime': '1100',
                                           'category': 'REH',
                                           'nx': 62,
                                           'ny': 123,
                                           'obsrValue': '35'},
                                          {'baseDate': '20231204',
                                           'baseTime': '1100',
                                           'category': 'RN1',
                                          

In [8]:
# 응답 데이터 정리
data = res.json() # json.loads(res.text)와 같은 기능
data = data['response']['body']['items']['item']
pprint(data)

# category 표
categorys = {
    'T1H':'기온',
    'RN1':'1시간 강수량',
    'UUU':'동서바람성분',
    'VVV':'남북바람성분',
    'REH':'습도',
    'PTY':'강수형태',
    'VEC':'풍향',
    'WSD':'풍속',
}

# 최종 데이터 생성
results = {}
for d in data:
    results[categorys[d['category']]] = d['obsrValue']
pprint(results)

[{'baseDate': '20231204',
  'baseTime': '1100',
  'category': 'PTY',
  'nx': 62,
  'ny': 123,
  'obsrValue': '0'},
 {'baseDate': '20231204',
  'baseTime': '1100',
  'category': 'REH',
  'nx': 62,
  'ny': 123,
  'obsrValue': '35'},
 {'baseDate': '20231204',
  'baseTime': '1100',
  'category': 'RN1',
  'nx': 62,
  'ny': 123,
  'obsrValue': '0'},
 {'baseDate': '20231204',
  'baseTime': '1100',
  'category': 'T1H',
  'nx': 62,
  'ny': 123,
  'obsrValue': '4.3'},
 {'baseDate': '20231204',
  'baseTime': '1100',
  'category': 'UUU',
  'nx': 62,
  'ny': 123,
  'obsrValue': '-0.6'},
 {'baseDate': '20231204',
  'baseTime': '1100',
  'category': 'VEC',
  'nx': 62,
  'ny': 123,
  'obsrValue': '120'},
 {'baseDate': '20231204',
  'baseTime': '1100',
  'category': 'VVV',
  'nx': 62,
  'ny': 123,
  'obsrValue': '0.4'},
 {'baseDate': '20231204',
  'baseTime': '1100',
  'category': 'WSD',
  'nx': 62,
  'ny': 123,
  'obsrValue': '0.8'}]
{'1시간 강수량': '0',
 '강수형태': '0',
 '기온': '4.3',
 '남북바람성분': '0.4',
 '동서바

In [9]:
# 완성코드
import requests
import pandas as pd
from datetime import datetime, timedelta

# 초단기예보 : /getUltraSrtFcst , 단기예보 : /getVilageFcst , 초단기실황 : getUltraSrtNcst
base_url = "http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst"
with open('keys/기상청api.txt') as file:
    service_key = file.read()       # 발급받은 기상청 API 키 입력

def get_weather(nx, ny):
    # 웹 요청할 base_date, base_time 계산
    now = datetime.now() # 현재 시각 데이터 생성

    # 40분 이전이면 현재 시보다 1시간 전 `base_time`을 요청한다.
    if now.minute <= 40:
        # 단. 00:40분 이전이라면 `base_date`는 전날이고 `base_time`은 2300이다.
        if now.hour == 0 :
            base_date = (now-timedelta(days=1)).strftime('%Y%m%d')
            base_time = '2300'
        else:
            base_date = now.strftime('%Y%m%d')
            base_time = (now-timedelta(hours=1)).strftime('%H00')
    # 40분 이후면 현재 시와 같은 `base_time`을 요청한다.
    else:
        base_date = base_date = now.strftime('%Y%m%d') 
        base_time = now.strftime('%H00')
    # print(base_date, base_time)


    # 웹 요청시 같이 전달될 데이터 = 요청 메시지
    params = {
        'serviceKey' : service_key,
        'numOfRows' : 30,
        'pageNo' : 1,
        'dataType' : 'JSON',
        'base_date' : base_date, # 오늘 날짜
        'base_time' : base_time, # 발표 시간
        'nx' : nx, # 62, # 성남시 운중동
        'ny' : ny, # 123 
    }

    res = requests.get(url=base_url , params=params)
    

    # 응답 데이터 정리
    from pprint import pprint  # 구조있는 데이터를 더 편하게 보여줌
    data = res.json() # json.loads(res.text)와 같은 기능
    data = data['response']['body']['items']['item']


    # category 표
    categorys = {
        'T1H':'기온',
        'RN1':'1시간 강수량',
        'UUU':'동서바람성분',
        'VVV':'남북바람성분',
        'REH':'습도',
        'PTY':'강수형태',
        'VEC':'풍향',
        'WSD':'풍속',
    }

    # 최종 데이터 생성
    results = {}
    for d in data:
        category = categorys[d['category']]
        value = d['obsrValue']
        # 각 카테고리에 따라 단위 추가
        if category == '기온':
            value = f"{value}°C"
        elif category == '1시간 강수량':
            value = f"{value}mm"
        elif category in ['동서바람성분', '남북바람성분', '풍속']:
            value = f"{value}m/s"
        elif category == '습도':
            value = f"{value}%"
        # 남북방향 기준으로 8방위 표시
        elif category == '풍향':
            value = int(value)
            if 0 <= value <= 45:
                value = "북동"
            elif 46 <= value <= 90:
                value = "동"
            elif 91 <= value <= 135:
                value = "남동"
            elif 136 <= value <= 180:
                value = "남"
            elif 181 <= value <= 225:
                value = "남서"
            elif 226 <= value <= 270:
                value = "서"
            elif 271 <= value <= 315:
                value = "북서"
            elif 316 <= value <= 360:
                value = "북"
        
        results[category] = value
    
    return results
    

# 테스트 코드
if __name__ == "__main__":
    print("성남시 운중동 날씨" , get_weather(62, 123))

성남시 운중동 날씨 {'강수형태': '0', '습도': '35%', '1시간 강수량': '0mm', '기온': '4.3°C', '동서바람성분': '-0.6m/s', '풍향': '남동', '남북바람성분': '0.4m/s', '풍속': '0.8m/s'}
